In [19]:
import os

In [30]:
%pwd

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project'

In [33]:
os.getcwd()

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project'

In [35]:
os.chdir("ocular-disease-recognition")

In [36]:
%pwd

'c:\\Users\\rohan\\Documents\\MSCS\\ECC\\Project\\ocular-disease-recognition'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [38]:
from src.ocularDiseaseRecognition.constants import *
from src.ocularDiseaseRecognition.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [47]:
import os
import urllib.request as request
import zipfile
from src.ocularDiseaseRecognition import logger
from src.ocularDiseaseRecognition.utils.common import get_size
import time
from requests.exceptions import Timeout
import subprocess

In [58]:
# the code is not able to download the file from git lfs

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self, timeout=30, max_retries=3, retry_delay=5):
        retry_count = 0
        while retry_count < max_retries:
            try:
                if not os.path.exists(self.config.local_data_file):
                    start_time = time.time()
                    response = request.urlopen(self.config.source_URL, timeout=timeout)
                    with open(self.config.local_data_file, 'wb') as f:
                        f.write(response.read())
                    elapsed_time = time.time() - start_time
                    logger.info(f"File downloaded successfully in {elapsed_time:.2f} seconds.")
                    return
                else:
                    logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
                    return
            except (Timeout, IOError) as e:
                logger.error(f"Error downloading file: {e}")
                retry_count += 1
                logger.info(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
        logger.error(f"Maximum retries reached. Unable to download file.")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [59]:
# code is not able to extract the zip file from the sha info obtained from git lfs

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Ensure the directory exists
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

        # Use Git LFS to pull the file
        try:
            subprocess.run(['git', 'lfs', 'pull', '-I', self.config.source_URL], check=True)
            print("File downloaded successfully from Git LFS.")
        except subprocess.CalledProcessError as e:
            print(f"Error downloading file from Git LFS: {e}")

    def extract_zip_file(self):
        # Ensure the directory exists
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        
        # Extract the zip file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            print("Zip file extracted successfully.")


In [60]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-18 12:30:56,624: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-18 12:30:56,626: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-18 12:30:56,627: INFO: common: created directory at: artifacts]
[2024-04-18 12:30:56,628: INFO: common: created directory at: artifacts/data_ingestion]
File downloaded successfully from Git LFS.


BadZipFile: File is not a zip file